In [3]:
import xgboost as xgb
import lightgbm

In [6]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [46]:
data=pd.read_csv('train_data.csv')
test=pd.read_csv('test_data.csv')
X=data.drop(['datetime','y','close'],axis=1)
y=data['y']
test_data=test.drop(['datetime','close'],axis=1)

In [47]:
X_train,X_test,y_train,y_test=train_test_split(X,y,shuffle=False)

In [57]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_test, label=y_test)
dtest = xgb.DMatrix(test_data)

In [58]:
lgb_results={}
params = {
        'objective': 'reg:pseudohubererror','silent':1, 'random_state':1234, 
        # 学習用の指標 (RMSE)
        'eval_metric': 'mae',
    }
num_round = 100
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]#訓練データはdtrain、評価用のテストデータはdvalidと設定
model = xgb.train(params,
                    dtrain,#訓練データ
                    num_round,#設定した学習回数
                    early_stopping_rounds=20,
                    evals=watchlist,
                  evals_result=lgb_results
                    )

[13:57:44] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-mae:12.38460	eval-mae:18.37878
[1]	train-mae:11.00855	eval-mae:16.92654
[2]	train-mae:10.54891	eval-mae:16.54074
[3]	train-mae:10.23693	eval-mae:16.22059
[4]	train-mae:10.12335	eval-mae:15.99000
[5]	train-mae:10.07609	eval-mae:15.93105
[6]	train-mae:10.04756	eval-mae:15.87304
[7]	train-mae:10.03113	eval-mae:15.77154
[8]	train-mae:10.01291	eval-mae:15.70699
[9]	train-mae:10.00197	eval-mae:15.62157
[10]	train-mae:9.98725	eval-mae:15.57683
[11]	train-mae:9.97479	eval-mae:15.49037
[12]	train-mae:9.96818	eval-mae:15.46069
[13]	train-mae:9.96362	eval-mae:

In [59]:
loss_train = lgb_results['train']['mae']  # 訓練誤差
loss_test = lgb_results['eval']['mae']    # 汎化誤差
best_iteration = model.best_iteration               # 最良の予測器が得られたイタレーション数
print(best_iteration)

60


In [61]:
tei=pd.DataFrame(model.predict(dtest,ntree_limit = model.best_ntree_limit))

In [64]:
tei.to_csv('xGboost.csv',index=None)

In [54]:
X.shape

(2101, 10)